In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc, rank, when
from pyspark.sql.window import Window

In [3]:
# 1. Initialize a Spark Session
spark = SparkSession.builder.appName("SchoolDashboard").getOrCreate()

In [4]:
df= spark.read.csv("/content/drive/MyDrive/Pyspark/Graincrop interview practice/dv402-SchoolLocations2025.csv", header=True, inferSchema=True)

In [5]:
df.printSchema()


root
 |-- Education_Sector: string (nullable = true)
 |-- Entity_Type: integer (nullable = true)
 |-- School_No: integer (nullable = true)
 |-- School_Name: string (nullable = true)
 |-- School_Type: string (nullable = true)
 |-- School_Status: string (nullable = true)
 |-- Address_Line_1: string (nullable = true)
 |-- Address_Line_2: string (nullable = true)
 |-- Address_Town: string (nullable = true)
 |-- Address_State: string (nullable = true)
 |-- Address_Postcode: integer (nullable = true)
 |-- Postal_Address_Line_1: string (nullable = true)
 |-- Postal_Address_Line_2: string (nullable = true)
 |-- Postal_Town: string (nullable = true)
 |-- Postal_State: string (nullable = true)
 |-- Postal_Postcode: integer (nullable = true)
 |-- Full_Phone_No: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Area: string (nullable = true)
 |-- LGA_ID: integer (nullable = true)
 |-- LGA_Name: string (nullable = true)
 |-- LGA_TYPE: string (nullable = true)
 |-- X: double (nulla

In [6]:

df.show(5)

+----------------+-----------+---------+--------------------+-----------+-------------+--------------------+--------------+--------------+-------------+----------------+---------------------+---------------------+--------------+------------+---------------+-------------+--------------------+--------------------+------+----------------+---------+----------+----------+
|Education_Sector|Entity_Type|School_No|         School_Name|School_Type|School_Status|      Address_Line_1|Address_Line_2|  Address_Town|Address_State|Address_Postcode|Postal_Address_Line_1|Postal_Address_Line_2|   Postal_Town|Postal_State|Postal_Postcode|Full_Phone_No|              Region|                Area|LGA_ID|        LGA_Name| LGA_TYPE|         X|         Y|
+----------------+-----------+---------+--------------------+-----------+-------------+--------------------+--------------+--------------+-------------+----------------+---------------------+---------------------+--------------+------------+---------------+---

In [7]:
 #Drop duplicates
df_spark = df.dropDuplicates()


In [9]:
from pyspark.sql import functions as F
# Count missing values in each column
missing = df_spark.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_spark.columns])
missing.show()

+----------------+-----------+---------+-----------+-----------+-------------+--------------+--------------+------------+-------------+----------------+---------------------+---------------------+-----------+------------+---------------+-------------+------+----+------+--------+--------+---+---+
|Education_Sector|Entity_Type|School_No|School_Name|School_Type|School_Status|Address_Line_1|Address_Line_2|Address_Town|Address_State|Address_Postcode|Postal_Address_Line_1|Postal_Address_Line_2|Postal_Town|Postal_State|Postal_Postcode|Full_Phone_No|Region|Area|LGA_ID|LGA_Name|LGA_TYPE|  X|  Y|
+----------------+-----------+---------+-----------+-----------+-------------+--------------+--------------+------------+-------------+----------------+---------------------+---------------------+-----------+------------+---------------+-------------+------+----+------+--------+--------+---+---+
|               0|          0|        0|          0|          0|            0|             0|          2289| 

In [10]:
# Total schools
total_schools = df_spark.select(F.countDistinct("School_No")).collect()[0][0]

In [12]:
print("Total Schools:", total_schools)

Total Schools: 2162


In [13]:
# Schools by Sector
sector_counts = (df_spark.groupBy("Education_Sector")
    .agg(F.countDistinct("School_No").alias("School_Count"))
    .orderBy(F.desc("School_Count")))

In [14]:
sector_counts.show()

+----------------+------------+
|Education_Sector|School_Count|
+----------------+------------+
|      Government|        1575|
|        Catholic|         493|
|     Independent|         233|
+----------------+------------+



In [15]:

# Schools by Type
type_counts = (df_spark.groupBy("School_Type")
    .agg(F.countDistinct("School_No").alias("School_Count"))
    .orderBy(F.desc("School_Count")))

In [17]:
type_counts.show()

+-----------+------------+
|School_Type|School_Count|
+-----------+------------+
|    Primary|        1491|
|  Secondary|         360|
|    Pri/Sec|         250|
|    Special|         117|
|   Language|           5|
+-----------+------------+



In [16]:

# Schools by Region
region_counts = (df_spark.groupBy("Region")
    .agg(F.countDistinct("School_No").alias("School_Count"))
    .orderBy(F.desc("School_Count")))

In [18]:
region_counts.show()

+--------------------+------------+
|              Region|School_Count|
+--------------------+------------+
|SOUTH-WESTERN VIC...|         631|
|SOUTH-EASTERN VIC...|         592|
|NORTH-EASTERN VIC...|         535|
|NORTH-WESTERN VIC...|         506|
+--------------------+------------+



In [19]:
summary = {
    "Total_Schools": total_schools,
    "Top_Sector": sector_counts.first().asDict(),
    "Top_Type": type_counts.first().asDict()
}
summary


{'Total_Schools': 2162,
 'Top_Sector': {'Education_Sector': 'Government', 'School_Count': 1575},
 'Top_Type': {'School_Type': 'Primary', 'School_Count': 1491}}

In [20]:
output_path = "/content/drive/MyDrive/schools_clean.csv"

(df_spark
   .coalesce(1)  # ensure a single CSV file instead of many parts
   .write.mode("overwrite")
   .option("header", True)
   .csv(output_path))


converting spark output to single csv file


In [21]:
import shutil, glob

# Path Spark wrote to (folder)
temp_path = "/content/drive/MyDrive/schools_clean.csv"

# Find the actual part file
part_file = glob.glob(temp_path + "/part-*.csv")[0]

# Copy it to a clean CSV file
final_path = "/content/drive/MyDrive/schools_clean_final.csv"
shutil.copy(part_file, final_path)

print("Final single CSV saved at:", final_path)


Final single CSV saved at: /content/drive/MyDrive/schools_clean_final.csv
